## A Deep Dive Into the Diversity of San Francisco Film Locations, 1915 - 2023

I moved to San Francisco in 2010. I love the city dearly and am proud to call it my home. Its reputation has suffered recently, in large part due to biased national news reporting. But perhaps the news isn’t wholly to blame for this misperception.

The longer I’ve lived here, the more I’ve come to realize that the San Francisco that’s so often portrayed in films and television tends to be limited to small pockets of the city. So perhaps the average American’s media-driven perception of how San Francisco “used to be” feeds into this overblown viewpoint of a city in decline. I decided to undergo an analysis of city filming locations to answer the following questions:

1. How have the number and scope of citywide filming locations changed over time?
2. How do the most-filmed districts correlate with actual San Francisco demographics?
3. Which titles offer the most diverse and balanced portrayals of San Francisco?
4. How does diversity of SF portrayal correlate to the most popular and profitable films and television shows?

Data sources: 
* SF Film Locations: DataSF, Open data project https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am/about_data 
* SF Supervisor District Demographics: https://www.sf.gov/departments/2020-census-redistricting-task-force 
* Income data from data.census.gov 
* Film budget and popularity: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies 

### Data Cleaning
The DataSF film locations dataset required extensive cleaning prior to conducting my analysis. While extremely comprehensive, the information was messy, inconsistent, and in some cases, incomplete. I took the following steps:

- Removed a handful of duplicate entries
- Removed non-SF locations (in Marin, East Bay, South Bay -- except SF-adjacent points of interest like Golden Gate Bridge, Alcatraz, and the Bay Bridge)
- Thorough standardization of locations
    - Cleaned up addresses
        - Converted entries with intersections to addresses
        - Converted entries with ranges of blocks to a single centered address
        - Split out entries with multiple locations noted
        - A few nonsensical locations deleted (ie - streets that don't intersect, streets that don't exist in the city)
    - Corrected street and landmark misspellings (ie - “Chrissy” Field, “Delores” Park)
    - Generated lat/long coordinates using the [Geocode for Sheets](https://workspace.google.com/marketplace/app/geocode_for_sheets_mapping_sheets/1083994590944) plugin
    - Adjusted location names to be uniform across multiple entries
- Standardized film/TV titles (in the case of television shows filming across multiple episodes and/or seasons)
- Input missing Supervisor District info where applicable
- Renamed columns to remove spaces (for easier SQL import)

I had to copy information from the 2020 redistricting report in order to build my table for SF district demographic data.

A quick inspection of the film database in SQL revealed the data to be in good shape for immediate analysis.

### Analysis
#### Part I: General Film Location Stats
Most popular filming locations
```
SELECT Locations,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_sf
GROUP BY Locations
ORDER BY n_films DESC;
```

Films with the most locations
```
SELECT Title, 
COUNT(DISTINCT Locations) as n_locations
FROM film_locations_sf
GROUP BY Title
ORDER BY n_locations DESC;
```

Films with the most “diverse” SF representation (filming locations across the most districts)
```
SELECT Title,
COUNT(DISTINCT CurrentSupervisorDistricts) as n_districts,
COUNT(DISTINCT Locations) AS n_locations
FROM film_locations_sf
GROUP BY Title
HAVING COUNT(DISTINCT CurrentSupervisorDistricts) > 4
ORDER BY n_districts DESC, n_locations DESC;
```

Films with the least "diverse" SF representation (least number of supervisor districts filmed in, relative to total number of filming locations)
```
SELECT Title,
COUNT(DISTINCT CurrentSupervisorDistricts) as n_districts,
COUNT(DISTINCT Locations) AS n_locations
FROM film_locations_sf
WHERE CurrentSupervisorDistricts IS NOT NULL
GROUP BY Title
HAVING COUNT(DISTINCT Locations) > 3
ORDER BY n_districts ASC, n_locations DESC;
```

Number of films by year, director, actor, writer, production company and distributor
```
SELECT ReleaseYear,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_sf
GROUP BY ReleaseYear;
```

```
SELECT Director,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_SF
GROUP BY Director
ORDER BY n_films DESC;
```

```
SELECT Writer,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_SF
GROUP BY Writer
ORDER BY n_films DESC;
```

```
SELECT actor,
COUNT(DISTINCT Title) as n_films
FROM
(select actor1 AS actor,
    Title
from film_locations_SF
UNION
SELECT actor2 AS actor,
    Title
FROM film_locations_SF
UNION
SELECT actor3 AS actor,
    Title
FROM film_locations_SF) t
WHERE actor IS NOT NULL
GROUP BY actor
ORDER BY n_films DESC;
```

```
SELECT productioncompany,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_sf
GROUP BY productioncompany
ORDER BY n_films DESC;
```

```
SELECT distributor,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_sf
GROUP BY distributor
ORDER BY n_films DESC;
```

Which supervisor district is the most popular for filming? Least popular?
```
SELECT CurrentSupervisorDistricts,
COUNT(DISTINCT Title) AS n_films
FROM film_locations_sf
GROUP BY CurrentSupervisorDistricts
ORDER BY n_films DESC;
```

#### Part II: Filming Location Data vs SF District Data
Are there any correlations between the number of films shot in each district and that district’s attributes/demographics? (TBD)

#### Part III: Filming Location Data vs Film Data
Joining the SF Film locations with the Kaggle film dataset
```
CREATE TABLE sf_film_stats AS
SELECT DISTINCT a.title, 
	b.imdb_id, 
	a.releaseyear, 
	b.release_date, 
	b.budget, 
	b.revenue, 
	b.popularity,
	b.vote_average,
	b.overview
FROM film_locations_sf a
LEFT JOIN tmdb_movie_dataset b
ON a.title = b.title
WHERE SUBSTRING(b.release_date, 1, 4) = CAST(a.releaseyear AS varchar)
OR SUBSTRING(b.release_date, 1, 4) = CAST((a.releaseyear + 1) AS varchar);
```

Creating a table with district diversity stats by title
```
CREATE TABLE sf_diversity AS
SELECT Title,
    COUNT(DISTINCT CurrentSupervisorDistricts) AS DistrictsDiversity
FROM film_locations_sf
GROUP BY Title;
```

Joining these new tables, filtering out duplicate titles with vote averages of 0
```
SELECT a.title,
	a.budget,
	a.revenue,
	a.popularity,
	a.vote_average,
	b.districtsdiversity,
	a.overview
FROM sf_film_stats a
JOIN sf_diversity b
ON a.title = b.title
WHERE vote_average != 0
ORDER BY popularity DESC;
```

### Key Insights

### Interactive Visualization
[HERE](https://public.tableau.com/app/profile/katelyn.mueller.mclean/viz/SanFranciscoFilmingLocations/Dashboard1#1)

